## Linear Regression Implementation from Scratch

In [1]:
%matplotlib inline
from d2l import torch as d2l
import torch 

#### Step 1: Initializing the model
Before starting SGD, we need to initialize the model

In [4]:
class LinearRegressionScratch(d2l.Module):  #@save
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.w = torch.normal(mean=0, std=sigma, size=(num_inputs, 1), requires_grad=True)
        self.b = torch.tensor([0.], requires_grad=True)

In [5]:
@d2l.add_to_class(LinearRegressionScratch)  #@save
def forward(self, X):
    return self.w @ X + self.b

#### Step 2: Defining the loss function
We are using the mean squared loss

In [6]:
@d2l.add_to_class(LinearRegressionScratch)  #@save
def loss(self, y_hat, y):
    l = (y_hat - y) ** 2 / 2
    return l.mean()

#### Step 3: Defining the optimization algorithm
we are using minibatch SGD. The following code applies the update, given a set of parameters, a learning rate lr

In [7]:
class SGD(d2l.HyperParameters):  #@save
    def __init__(self, params, lr):
        self.save_hyperparameters()

    def step(self):
        for param in self.params:
            param += (-1) * param.grad * self.lr

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

In [8]:
@d2l.add_to_class(LinearRegressionScratch)  #@save
def configure_optimizers(self):
    return SGD([self.w, self.b], self.lr)

#### Step 4: Training

In [9]:
@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def fit_epoch(self):
    self.model.train()
    for batch in self.train_dataloader:
        loss = self.model.training_step(self.prepare_batch(batch))
        self.optim.zero_grad()
        with torch.no_grad():
            loss.backward()
            if self.gradient_clip_val > 0:
                self.clip_gradients(self.gradient_clip_val, self.model)
            self.optim.step()
        self.train_batch_idx += 1
    if self.val_dataloader is None:
        return
    self.model.eval()
    for batch in self.val_dataloader:
        with torch.no_grad():
            self.model.validation_step(self.prepare_batch(batch))
        self.val_batch_idx += 1